In [24]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': '****',         # имя пользователя
             'pwd': '****', # пароль
             'host': '****',
             'port': 5432,              # порт подключения
             'db': '****'}          # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string)

# 1. Посчитайте затраты на каждую рекламную кампанию.

In [32]:
query = '''
SELECT
    source_id,
    sum(costs) as sum
FROM 
    advertisment_costs
GROUP BY
    source_id
'''
pd.io.sql.read_sql(query, con = engine)

,source_id,sum
0,10,"$5,822.49"
1,9,"$5,517.49"
2,1,"$20,833.27"
3,5,"$51,757.10"
4,4,"$61,073.60"
5,2,"$42,806.04"
6,3,"$141,321.63"


# 2. Посчитайте суммарную выручку и размах (разницу между максимальным и минимальным значением) от заказов, сделанных с настольных ПК (тип устройства desktop).

In [33]:
query = '''
SELECT
    sum(revenue) AS sum,
    max(revenue) - min(revenue) AS scope
FROM 
    orders o left join visits v on o.Uid = v.UId
WHERE
    v.Device = 'desktop'
'''
pd.io.sql.read_sql(query, con = engine)

,sum,scope
0,"$476,625.95","$2,633.28"


# 3. Определите эффективность рекламной кампании в зависимости от категории устройства пользователя.

In [55]:
query = '''
WITH 
o AS (SELECT
    uid,
    buyts::date AS datets,
    sum(revenue) AS sum_r
FROM 
    orders
GROUP BY
    uid,
    buyts::date),
    
v AS (SELECT
    uid,
    device,
    sourceid,
    startts::date AS date_start,
    endts::date AS date_end
FROM 
    visits),

c AS (SELECT
    source_id,
    sum(costs) AS sum_coast,
    dt::date AS date_costs
FROM 
    advertisment_costs
GROUP BY
    source_id,
    dt::date)

SELECT
    c.date_costs AS date,
    o.sum_r - c.sum_coast AS efficiency
FROM
    v
LEFT JOIN o ON o.uid = v.uid AND o.datets = v.date_end
FULL OUTER JOIN c ON c.source_id = v.sourceid AND c.date_costs = v.date_end  
WHERE
    o.sum_r - c.sum_coast IS NOT NULL
ORDER BY
    date
'''
pd.io.sql.read_sql(query, con = engine)

,date,efficiency
0,2017-06-01,-$126.46
1,2017-06-01,-$74.83
2,2017-06-01,-$347.30
3,2017-06-01,-$58.20
4,2017-06-01,-$70.98
5,2017-06-01,-$128.83
6,2017-06-01,-$131.34
7,2017-06-01,-$131.03
8,2017-06-01,-$73.98
9,2017-06-02,-$56.77
